In [1]:
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
import pandas as pd
import plotly.graph_objects as go
import scipy
from matplotlib.lines import Line2D
import networkx as nx
from tqdm import tqdm
import matplotlib.colors as mcolors
from sklearn.linear_model import LinearRegression
from scipy.optimize import curve_fit
import psycopg2


axis_label_fontsize = 12
title_fontsize = 15
pd.set_option("display.max_columns",100)

In [2]:
df_elec_price = pd.read_csv("data/electricity_prices_ANNUAL_Hungary_EUROSTAT_2018-2023.csv")
df_elec_price = df_elec_price.rename(columns = {"lower_bound_[currency]":"lower_bound_HUF", 
                                        "upper_bound_[currency]":"upper_bound_HUF"})

df_gas_price = pd.read_csv("data/gas_prices_ANNUAL_Hungary_EUROSTAT_2018-2023.csv")
df_gas_price = df_gas_price.rename(columns = {"lower_bound_[currency]":"lower_bound_HUF", 
                                        "upper_bound_[currency]":"upper_bound_HUF"})

df_fuel_price = pd.read_csv("data/fuel_price_Hungary_2018-2023.csv")




In [3]:
df = pd.read_csv("data/energy_purchases_revenue_firms_2020-2023.csv")

df["fossil_purchases_2020"] = df["gas_purchases_2020"] + df["oil_purchases_2020"]
df["fossil_purchases_2021"] = df["gas_purchases_2021"] + df["oil_purchases_2021"]
df["fossil_purchases_2022"] = df["gas_purchases_2022"] + df["oil_purchases_2022"]
df["fossil_purchases_2023"] = df["gas_purchases_2023"] + df["oil_purchases_2023"]

df

,firm_id,oil_purchases_2020,gas_purchases_2020,electricity_purchases_2020,oil_purchases_2021,gas_purchases_2021,electricity_purchases_2021,oil_purchases_2022,gas_purchases_2022,electricity_purchases_2022,oil_purchases_2023,gas_purchases_2023,electricity_purchases_2023,revenue_2020,revenue_2021,revenue_2022,revenue_2023,nace2,nace4,parent_nace,fossil_purchases_2020,fossil_purchases_2021,fossil_purchases_2022,fossil_purchases_2023
0,1,0,0,0,0,0,0,0,0,0,0,0,0,3259982,3.156703e+06,3.179511e+06,3.240726e+06,19,1920,C,0,0,0,0
1,2,0,0,0,0,0,0,0,0,0,0,0,0,938371,9.464891e+05,9.202791e+05,9.410088e+05,35,3523,D,0,0,0,0
2,3,0,0,0,0,0,0,0,0,0,0,0,0,3137966,2.987377e+06,3.122631e+06,3.123044e+06,35,3511,D,0,0,0,0
3,4,6541377,1305711,2484405,8534258,2419224,1556209,8071492,1684007,1716713,6538112,2793119,1826565,918217,9.484208e+05,9.434061e+05,9.864333e+05,24,2410,C,7847088,10953482,9755499,9331231
4,5,8771495,1117952,1963395,6410807,1230131,2966877,7917999,1354361,2128739,7308554,2666654,2810688,4743876,4.508910e+06,4.665102e+06,4.732275e+06,7,710,B,9889447,7640938,9272360,9975208
5,6,8467927,1152315,2930947,6320290,2098387,1724115,7259508,2322854,1978039,8492786,2234801,1233138,1709814,1.740217e+06,1.774931e+06,1.761416e+06,42,4221,F,9620242,8418677,9582362,10727587
6,7,0,8487139,2352713,0,8340864,2794471,0,7367217,2810862,0,6854483,2697698,1000139,9.771366e+05,9.573434e+05,9.675289e+05,17,1711,C,8487139,8340864,7367217,6854483
7,8,0,7300890,2659157,0,8320356,2232137,0,8234304,1086107,0,6802236,2654621,2441634,2.499060e+06,2.577480e+06,2.453553e+06,42,4221,F,7300890,8320356,8234304,6802236
8,9,0,8554836,1374564,0,8359705,2430143,0,7729580,2755747,0,8926117,2672802,4130771,4.321691e+06,4.276963e+06,4.192098e+06,25,2511,C,8554836,8359705,7729580,8926117
9,10,0,1735831,9398847,0,2352224,7245707,0,2844789,6827602,0,2490474,9264812,3062512,2.985567e+06,3.099348e+06,3.148992e+06,52,5221,H,1735831,2352224,2844789,2490474


In [4]:
df_filtered = df[
## Every firm needs to have a gas and an electricity link in every year
   (df["electricity_purchases_2020"]>0) 
  & 
    (df["electricity_purchases_2021"]>0)
  & 
    (df["electricity_purchases_2022"]>0)
  & 
    (df["electricity_purchases_2023"]>0)
  &
    (df["gas_purchases_2020"]>0)
  & 
    (df["gas_purchases_2021"]>0)
  &
    (df["gas_purchases_2022"]>0)
  &
    (df["gas_purchases_2023"]>0)
  &

### Sector exclusions
## Electricity & Gas
   (df.nace2!=35)
  &
## Oil
    (df.nace4!=4730)
  &
    (df.nace4!=4671)
  &
    (df.nace4!=1920)
  &
    (df.nace4!=610)
  &

## Financial sector
    (df.nace2!=64) 
  &
    (df.nace2!=65)
  &
    (df.nace2!=66)
  &

## Exclude firms without NACE code
    (df.nace2>0)
  &

## Exclude firms with no consistent revenue [2020-2022]
    (df.revenue_2020>0)
  &
    (df.revenue_2021>0)
  &
    (df.revenue_2022>0)
  &
    (df.revenue_2023>0)
]

df_filtered = df_filtered.reset_index(drop=True)
df_filtered

,firm_id,oil_purchases_2020,gas_purchases_2020,electricity_purchases_2020,oil_purchases_2021,gas_purchases_2021,electricity_purchases_2021,oil_purchases_2022,gas_purchases_2022,electricity_purchases_2022,oil_purchases_2023,gas_purchases_2023,electricity_purchases_2023,revenue_2020,revenue_2021,revenue_2022,revenue_2023,nace2,nace4,parent_nace,fossil_purchases_2020,fossil_purchases_2021,fossil_purchases_2022,fossil_purchases_2023
0,4,6541377,1305711,2484405,8534258,2419224,1556209,8071492,1684007,1716713,6538112,2793119,1826565,918217,9.484208e+05,9.434061e+05,9.864333e+05,24,2410,C,7847088,10953482,9755499,9331231
1,5,8771495,1117952,1963395,6410807,1230131,2966877,7917999,1354361,2128739,7308554,2666654,2810688,4743876,4.508910e+06,4.665102e+06,4.732275e+06,7,710,B,9889447,7640938,9272360,9975208
2,6,8467927,1152315,2930947,6320290,2098387,1724115,7259508,2322854,1978039,8492786,2234801,1233138,1709814,1.740217e+06,1.774931e+06,1.761416e+06,42,4221,F,9620242,8418677,9582362,10727587
3,7,0,8487139,2352713,0,8340864,2794471,0,7367217,2810862,0,6854483,2697698,1000139,9.771366e+05,9.573434e+05,9.675289e+05,17,1711,C,8487139,8340864,7367217,6854483
4,8,0,7300890,2659157,0,8320356,2232137,0,8234304,1086107,0,6802236,2654621,2441634,2.499060e+06,2.577480e+06,2.453553e+06,42,4221,F,7300890,8320356,8234304,6802236
5,9,0,8554836,1374564,0,8359705,2430143,0,7729580,2755747,0,8926117,2672802,4130771,4.321691e+06,4.276963e+06,4.192098e+06,25,2511,C,8554836,8359705,7729580,8926117
6,10,0,1735831,9398847,0,2352224,7245707,0,2844789,6827602,0,2490474,9264812,3062512,2.985567e+06,3.099348e+06,3.148992e+06,52,5221,H,1735831,2352224,2844789,2490474
7,11,0,1310744,8954386,0,2884874,6899350,0,2600121,8580048,0,1217456,7717211,1575712,1.587712e+06,1.600616e+06,1.567015e+06,45,4531,G,1310744,2884874,2600121,1217456
8,12,0,1839052,6956463,0,2958039,6381981,0,1536660,7888779,0,1024777,6536076,4645318,4.688074e+06,4.867035e+06,4.924473e+06,42,4221,F,1839052,2958039,1536660,1024777


## Estimate Electricity, Gas and Oil consumption for every firm for every year between 2020-2023 using energy prices [for Hungary]

In [5]:
def closest(lst, K):
    return lst[min(range(len(lst)), key = lambda i: abs(lst[i]-K))]
     

def find_price(val, bounds, energy_type, df_temp):
    
    price = np.nan
    
    for i in range(len(bounds)):
        bound = bounds[i]
        if ((bound[0] <= val) and (val <= bound[1])): 
                price = df_temp.loc[i,"{}_price_per_MWh".format(energy_type)]
                #print(bound, val, price)

    if np.isnan(price): 
         #print(val)
         temp_bounds = list(np.array([list(i) for i in bounds]).flat)
         closest_value = closest(temp_bounds, val)
         price = df_temp[df_temp.isin([closest_value]).any(axis=1)]["{}_price_per_MWh".format(energy_type)].values[0]

    return val/price, price


In [6]:
for year in np.arange(2020,2024):

    df_elec_prices_temp = df_elec_price[df_elec_price.year == year].reset_index()
    elec_bounds = list(zip(df_elec_prices_temp.loc[:,"lower_bound_HUF"], df_elec_prices_temp.loc[:,"upper_bound_HUF"]))

    df_gas_prices_temp = df_gas_price[df_gas_price.year == year].reset_index()
    gas_bounds = list(zip(df_gas_prices_temp.loc[:,"lower_bound_HUF"], df_gas_prices_temp.loc[:,"upper_bound_HUF"]))

    for i in tqdm(range(len(df_filtered))):
        ## ELECTRICITY
        elec_val_HUF = df_filtered.loc[i, "electricity_purchases_{}".format(year)]
        if elec_val_HUF > 0:
            elec_val_MWH, elec_price = find_price(elec_val_HUF,elec_bounds,"elec",df_elec_prices_temp)
            df_filtered.loc[i,"elec_consumption_KWH_{}".format(year)] = elec_val_MWH*1000 ## convert to KWH
            df_filtered.loc[i,"elec_price_{}".format(year)] = elec_price
        else: 
            df_filtered.loc[i,"elec_consumption_KWH_{}".format(year)] = 0
        ## GAS
        gas_val_HUF = df_filtered.loc[i, "gas_purchases_{}".format(year)]
        if gas_val_HUF > 0:
            gas_val_MWH, gas_price = find_price(gas_val_HUF,gas_bounds,"gas",df_gas_prices_temp)
            df_filtered.loc[i,"gas_consumption_KWH_{}".format(year)] = gas_val_MWH*1000 ## convert to KWH
            df_filtered.loc[i,"gas_price_{}".format(year)] = gas_price
        else: 
            df_filtered.loc[i,"gas_consumption_KWH_{}".format(year)] = 0
        ## OIL
        oil_val_HUF = df_filtered.loc[i, "oil_purchases_{}".format(year)]
        if oil_val_HUF > 0:
            oil_price = df_fuel_price[df_fuel_price.year==year]["fuel_price_HUF"].values[0]
            oil_val_KWH = (oil_val_HUF/oil_price)*9.6 ## 1l diesel = 10 kWh, 1l gasoline = 8.9 kWh
            df_filtered.loc[i,"oil_consumption_KWH_{}".format(year)] = oil_val_KWH
        else: 
            df_filtered.loc[i,"oil_consumption_KWH_{}".format(year)] = 0

  0%|          | 0/9 [00:00<?, ?it/s]

100%|██████████| 9/9 [00:00<00:00, 181.91it/s]


In [7]:
## Calculate electricity share for every year 2020-2023

df_filtered["elec_share_2020"] = df_filtered["elec_consumption_KWH_2020"]/(df_filtered["elec_consumption_KWH_2020"] + df_filtered["gas_consumption_KWH_2020"] + df_filtered["oil_consumption_KWH_2020"])
df_filtered["elec_share_2021"] = df_filtered["elec_consumption_KWH_2021"]/(df_filtered["elec_consumption_KWH_2021"] + df_filtered["gas_consumption_KWH_2021"] + df_filtered["oil_consumption_KWH_2021"])
df_filtered["elec_share_2022"] = df_filtered["elec_consumption_KWH_2022"]/(df_filtered["elec_consumption_KWH_2022"] + df_filtered["gas_consumption_KWH_2022"] + df_filtered["oil_consumption_KWH_2022"])
df_filtered["elec_share_2023"] = df_filtered["elec_consumption_KWH_2023"]/(df_filtered["elec_consumption_KWH_2023"] + df_filtered["gas_consumption_KWH_2023"] + df_filtered["oil_consumption_KWH_2023"])

df_filtered

,firm_id,oil_purchases_2020,gas_purchases_2020,electricity_purchases_2020,oil_purchases_2021,gas_purchases_2021,electricity_purchases_2021,oil_purchases_2022,gas_purchases_2022,electricity_purchases_2022,oil_purchases_2023,gas_purchases_2023,electricity_purchases_2023,revenue_2020,revenue_2021,revenue_2022,revenue_2023,nace2,nace4,parent_nace,fossil_purchases_2020,fossil_purchases_2021,fossil_purchases_2022,fossil_purchases_2023,elec_consumption_KWH_2020,elec_price_2020,gas_consumption_KWH_2020,gas_price_2020,oil_consumption_KWH_2020,elec_consumption_KWH_2021,elec_price_2021,gas_consumption_KWH_2021,gas_price_2021,oil_consumption_KWH_2021,elec_consumption_KWH_2022,elec_price_2022,gas_consumption_KWH_2022,gas_price_2022,oil_consumption_KWH_2022,elec_consumption_KWH_2023,elec_price_2023,gas_consumption_KWH_2023,gas_price_2023,oil_consumption_KWH_2023,elec_share_2020,elec_share_2021,elec_share_2022,elec_share_2023
0,4,6541377,1305711,2484405,8534258,2419224,1556209,8071492,1684007,1716713,6538112,2793119,1826565,918217,9.484208e+05,9.434061e+05,9.864333e+05,24,2410,C,7847088,10953482,9755499,9331231,49836.813050,49850.8,102389.432577,12752.4,171136.037929,30366.149802,51248.15,171629.728426,14095.60,183712.693014,16569.741123,103605.30,63204.104497,26643.95,138302.597634,13560.229888,134700.15,56480.616832,49452.7,102001.412433,0.154121,0.078728,0.075981,0.078819
1,5,8771495,1117952,1963395,6410807,1230131,2966877,7917999,1354361,2128739,7308554,2666654,2810688,4743876,4.508910e+06,4.665102e+06,4.732275e+06,7,710,B,9889447,7640938,9272360,9975208,39385.426112,49850.8,87666.007967,12752.4,229480.566708,57892.372700,51248.15,87270.566702,14095.60,138002.228005,21249.427897,100178.65,50831.839874,26643.95,135672.541058,20866.257387,134700.15,53923.324712,49452.7,114021.116622,0.110468,0.204447,0.102282,0.110514
2,6,8467927,1152315,2930947,6320290,2098387,1724115,7259508,2322854,1978039,8492786,2234801,1233138,1709814,1.740217e+06,1.774931e+06,1.761416e+06,42,4221,F,9620242,8418677,9582362,10727587,58794.382437,49850.8,90360.637998,12752.4,221538.595964,33642.482704,51248.15,148868.228383,14095.60,136053.713929,19092.063823,103605.30,87181.292564,26643.95,124389.495021,9154.689137,134700.15,45190.677152,49452.7,132496.379305,0.158606,0.105607,0.082770,0.048997
3,7,0,8487139,2352713,0,8340864,2794471,0,7367217,2810862,0,6854483,2697698,1000139,9.771366e+05,9.573434e+05,9.675289e+05,17,1711,C,8487139,8340864,7367217,6854483,47195.090149,49850.8,697393.465792,12169.8,0.000000,54528.231751,51248.15,504810.881966,16522.75,0.000000,28058.493501,100178.65,276506.186207,26643.95,0.000000,20027.431298,134700.15,138606.850587,49452.7,0.000000,0.063384,0.097487,0.092127,0.126249
4,8,0,7300890,2659157,0,8320356,2232137,0,8234304,1086107,0,6802236,2654621,2441634,2.499060e+06,2.577480e+06,2.453553e+06,42,4221,F,7300890,8320356,8234304,6802236,53342.313463,49850.8,599918.651087,12169.8,0.000000,43555.464929,51248.15,503569.684223,16522.75,0.000000,10483.122002,103605.30,309049.671689,26643.95,0.000000,19707.632100,134700.15,137550.346088,49452.7,0.000000,0.081655,0.079608,0.032808,0.125320
5,9,0,8554836,1374564,0,8359705,2430143,0,7729580,2755747,0,8926117,2672802,4130771,4.321691e+06,4.276963e+06,4.192098e+06,25,2511,C,8554836,8359705,7729580,8926117,27573.559502,49850.8,702956.170192,12169.8,0.000000,47419.136105,51248.15,505951.188513,16522.75,0.000000,27508.326375,100178.65,290106.384376,26643.95,0.000000,19842.605966,134700.15,180498.071895,49452.7,0.000000,0.037745,0.085692,0.086609,0.099044
6,10,0,1735831,9398847,0,2352224,7245707,0,2844789,6827602,0,2490474,9264812,3062512,2.985567e+06,3.099348e+06,3.148992e+06,52,5221,H,1735831,2352224,2844789,2490474,188539.541993,49850.8,136117.985634,12752.4,0.000000,141384.752425,51248.15,166876.472091,14095.60,0.000000,68154.262410,100178.65,106770.542656,26643.95,0.000000,62164.843172,149036.20,50360.728535,49452.7,0.000000,0.580734,0.458652,0.389620,0.552451
7,11,0,1310744,8954386,0,2884874,6899350

## Calculate low-carbon electricity share of every firm

In [8]:
## low-carbon share of the Hungarian electricity mix, derived from https://ember-energy.org/data/electricity-data-explorer/
df_lowcarbon_share_Hungary = pd.read_csv("data/low-carbon_electricity_share_Hungary_[2020-2023].csv")
df_lowcarbon_share_Hungary


df_filtered["total_consumption_KWH_2020"] = (df_filtered["elec_consumption_KWH_2020"] + df_filtered["gas_consumption_KWH_2020"] + df_filtered["oil_consumption_KWH_2020"])
df_filtered["total_consumption_KWH_2021"] = (df_filtered["elec_consumption_KWH_2021"] + df_filtered["gas_consumption_KWH_2021"] + df_filtered["oil_consumption_KWH_2021"])
df_filtered["total_consumption_KWH_2022"] = (df_filtered["elec_consumption_KWH_2022"] + df_filtered["gas_consumption_KWH_2022"] + df_filtered["oil_consumption_KWH_2022"])
df_filtered["total_consumption_KWH_2023"] = (df_filtered["elec_consumption_KWH_2023"] + df_filtered["gas_consumption_KWH_2023"] + df_filtered["oil_consumption_KWH_2023"])
df_filtered["avg_total_consumption_KWH"] = 1/4*(df_filtered["total_consumption_KWH_2020"] + df_filtered["total_consumption_KWH_2021"] + df_filtered["total_consumption_KWH_2022"] + df_filtered["total_consumption_KWH_2023"])

df_filtered["low-carbon_elec_consumption_KWH_2020"] = df_filtered["elec_consumption_KWH_2020"] * df_lowcarbon_share_Hungary[df_lowcarbon_share_Hungary.year==2020]["low-carbon_elec_share"].values[0]
df_filtered["low-carbon_elec_consumption_KWH_2021"] = df_filtered["elec_consumption_KWH_2021"] * df_lowcarbon_share_Hungary[df_lowcarbon_share_Hungary.year==2021]["low-carbon_elec_share"].values[0]
df_filtered["low-carbon_elec_consumption_KWH_2022"] = df_filtered["elec_consumption_KWH_2022"] * df_lowcarbon_share_Hungary[df_lowcarbon_share_Hungary.year==2022]["low-carbon_elec_share"].values[0]
df_filtered["low-carbon_elec_consumption_KWH_2023"] = df_filtered["elec_consumption_KWH_2023"] * df_lowcarbon_share_Hungary[df_lowcarbon_share_Hungary.year==2023]["low-carbon_elec_share"].values[0]

df_filtered["low-carbon_elec_share_2020"] = df_filtered["low-carbon_elec_consumption_KWH_2020"]/(df_filtered["elec_consumption_KWH_2020"] + df_filtered["gas_consumption_KWH_2020"] + df_filtered["oil_consumption_KWH_2020"])
df_filtered["low-carbon_elec_share_2021"] = df_filtered["low-carbon_elec_consumption_KWH_2021"]/(df_filtered["elec_consumption_KWH_2021"] + df_filtered["gas_consumption_KWH_2021"] + df_filtered["oil_consumption_KWH_2021"])
df_filtered["low-carbon_elec_share_2022"] = df_filtered["low-carbon_elec_consumption_KWH_2022"]/(df_filtered["elec_consumption_KWH_2022"] + df_filtered["gas_consumption_KWH_2022"] + df_filtered["oil_consumption_KWH_2022"])
df_filtered["low-carbon_elec_share_2023"] = df_filtered["low-carbon_elec_consumption_KWH_2023"]/(df_filtered["elec_consumption_KWH_2023"] + df_filtered["gas_consumption_KWH_2023"] + df_filtered["oil_consumption_KWH_2023"])

df_filtered["fossil_share_2020"] = 1-df_filtered["low-carbon_elec_share_2020"]
df_filtered["fossil_share_2021"] = 1-df_filtered["low-carbon_elec_share_2021"]
df_filtered["fossil_share_2022"] = 1-df_filtered["low-carbon_elec_share_2022"]
df_filtered["fossil_share_2023"] = 1-df_filtered["low-carbon_elec_share_2023"]
df_filtered

,firm_id,oil_purchases_2020,gas_purchases_2020,electricity_purchases_2020,oil_purchases_2021,gas_purchases_2021,electricity_purchases_2021,oil_purchases_2022,gas_purchases_2022,electricity_purchases_2022,oil_purchases_2023,gas_purchases_2023,electricity_purchases_2023,revenue_2020,revenue_2021,revenue_2022,revenue_2023,nace2,nace4,parent_nace,fossil_purchases_2020,fossil_purchases_2021,fossil_purchases_2022,fossil_purchases_2023,elec_consumption_KWH_2020,elec_price_2020,gas_consumption_KWH_2020,gas_price_2020,oil_consumption_KWH_2020,elec_consumption_KWH_2021,elec_price_2021,gas_consumption_KWH_2021,gas_price_2021,oil_consumption_KWH_2021,elec_consumption_KWH_2022,elec_price_2022,gas_consumption_KWH_2022,gas_price_2022,oil_consumption_KWH_2022,elec_consumption_KWH_2023,elec_price_2023,gas_consumption_KWH_2023,gas_price_2023,oil_consumption_KWH_2023,elec_share_2020,elec_share_2021,elec_share_2022,elec_share_2023,total_consumption_KWH_2020,total_consumption_KWH_2021,total_consumption_KWH_2022,total_consumption_KWH_2023,avg_total_consumption_KWH,low-carbon_elec_consumption_KWH_2020,low-carbon_elec_consumption_KWH_2021,low-carbon_elec_consumption_KWH_2022,low-carbon_elec_consumption_KWH_2023,low-carbon_elec_share_2020,low-carbon_elec_share_2021,low-carbon_elec_share_2022,low-carbon_elec_share_2023,fossil_share_2020,fossil_share_2021,fossil_share_2022,fossil_share_2023
0,4,6541377,1305711,2484405,8534258,2419224,1556209,8071492,1684007,1716713,6538112,2793119,1826565,918217,9.484208e+05,9.434061e+05,9.864333e+05,24,2410,C,7847088,10953482,9755499,9331231,49836.813050,49850.8,102389.432577,12752.4,171136.037929,30366.149802,51248.15,171629.728426,14095.60,183712.693014,16569.741123,103605.30,63204.104497,26643.95,138302.597634,13560.229888,134700.15,56480.616832,49452.7,102001.412433,0.154121,0.078728,0.075981,0.078819,323362.283556,385708.571243,218076.443254,172042.259153,274797.389302,30798.351249,19262.737686,10875.051011,9514.887564,0.095244,0.049941,0.049868,0.055306,0.904756,0.950059,0.950132,0.944694
1,5,8771495,1117952,1963395,6410807,1230131,2966877,7917999,1354361,2128739,7308554,2666654,2810688,4743876,4.508910e+06,4.665102e+06,4.732275e+06,7,710,B,9889447,7640938,9272360,9975208,39385.426112,49850.8,87666.007967,12752.4,229480.566708,57892.372700,51248.15,87270.566702,14095.60,138002.228005,21249.427897,100178.65,50831.839874,26643.95,135672.541058,20866.257387,134700.15,53923.324712,49452.7,114021.116622,0.110468,0.204447,0.102282,0.110514,356532.000787,283165.167406,207753.808828,188810.698722,259065.418936,24339.561726,36723.970494,13946.422616,14641.351552,0.068268,0.129691,0.067130,0.077545,0.931732,0.870309,0.932870,0.922455
2,6,8467927,1152315,2930947,6320290,2098387,1724115,7259508,2322854,1978039,8492786,2234801,1233138,1709814,1.740217e+06,1.774931e+06,1.761416e+06,42,4221,F,9620242,8418677,9582362,10727587,58794.382437,49850.8,90360.637998,12752.4,221538.595964,33642.482704,51248.15,148868.228383,14095.60,136053.713929,19092.063823,103605.30,87181.292564,26643.95,124389.495021,9154.689137,134700.15,45190.677152,49452.7,132496.379305,0.158606,0.105607,0.082770,0.048997,370693.616398,318564.425017,230662.851408,186841.745594,276690.659604,36333.985481,21341.076286,12530.501620,6423.625451,0.098016,0.066991,0.054324,0.034380,0.901984,0.933009,0.945676,0.965620
3,7,0,8487139,2352713,0,8340864,2794471,0,7367217,2810862,0,6854483,2697698,1000139,9.771366e+05,9.573434e+05,9.675289e+05,17,1711,C,8487139,8340864,7367217,6854483,47195.090149,49850.8,697393.465792,12169.8,0.000000,54528.231751,51248.15,504810.881966,16522.75,0.000000,28058.493501,100178.65,276506.186207,26643.95,0.000000,20027.431298,134700.15,138606.850587,49452.7,0.000000,0.063384,0.097487,0.092127,0.126249,744588.555941,559339.113717,304564.679709,158634.281885,441781.657813,29165.808861,34589.930944,18415.347944,14052.767436,0.039170,0.061841,0.060464,0.088586,0.960830,0.938159,0.939536,0.911414
4,8,0,7300890,2659157,0,8320356,2232137,0,8234304,1086

## Estimate Decarbonization Trend

In [9]:
def estimate_decarbonization_trend(df_temp, index): 
    lowcarbon_elec_vals = df_temp.loc[index,[
                                "low-carbon_elec_share_2020",
                                "low-carbon_elec_share_2021",
                                "low-carbon_elec_share_2022",
                                "low-carbon_elec_share_2023",
                                ]].values

    years = np.arange(0,4).reshape((-1,1))
    lowcarbon_elec_model = LinearRegression()
    lowcarbon_elec_model.fit(years,lowcarbon_elec_vals)
    lowcarbon_elec_coef = lowcarbon_elec_model.coef_
    lowcarbon_elec_intercept = lowcarbon_elec_model.intercept_


    return lowcarbon_elec_coef, lowcarbon_elec_intercept
    

for i in tqdm(range(len(df_filtered))):
    elec_coef,elec_intercept = estimate_decarbonization_trend(df_filtered,i)
    df_filtered.loc[i,"decarbonization_trend"] = elec_coef
    df_filtered.loc[i,"decarbonization_trend_intercept"] = elec_intercept

100%|██████████| 9/9 [00:00<00:00, 85.17it/s]


## Estimate Decarbonization Rate

In [11]:
def exponential_model(x, a, b):
    return a * np.exp(b * x)

def estimate_decarbonization_rate(df_temp,index):

    lowcarbon_elec_vals = df_temp.loc[index,[
                            "low-carbon_elec_share_2020",
                            "low-carbon_elec_share_2021",
                            "low-carbon_elec_share_2022",
                            "low-carbon_elec_share_2023",
                            ]].values
    
    years = np.arange(0,4)
    try:
        params, covariance = curve_fit(exponential_model, years, lowcarbon_elec_vals, maxfev=10000)
        decarbonization_rate_initial_value, decarbonization_rate = params
    except:
        decarbonization_rate_initial_value, decarbonization_rate = np.nan, np.nan
    
    return decarbonization_rate_initial_value, decarbonization_rate

## REPLACE WITH df_filtered for re-run!
for i in tqdm(range(len(df_filtered))):
    decarbonization_rate_initial_value, decarbonization_rate = estimate_decarbonization_rate(df_filtered,i)
    df_filtered.loc[i,"decarbonization_rate"] = decarbonization_rate
    df_filtered.loc[i,"decarbonization_rate_initial_value"] = decarbonization_rate_initial_value

100%|██████████| 9/9 [00:00<00:00, 229.17it/s]


## Estimate LINEAR ELECTRICITY SHARE Growth
#### (required for scenarios)

In [12]:
def estimate_elec_share_growth(df_temp, index): 
    elec_vals = df_temp.loc[index,[
                                "elec_share_2020",
                                "elec_share_2021",
                                "elec_share_2022",
                                "elec_share_2023",
                                ]].values

    years = np.arange(0,4).reshape((-1,1))
    elec_model = LinearRegression()
    elec_model.fit(years,elec_vals)
    elec_coef = elec_model.coef_
    elec_intercept = elec_model.intercept_


    return elec_coef, elec_intercept

for i in tqdm(range(len(df_filtered))):
    elec_coef,elec_intercept = estimate_elec_share_growth(df_filtered,i)
    df_filtered.loc[i,"elec_trend"] = elec_coef
    df_filtered.loc[i,"elec_intercept"] = elec_intercept

100%|██████████| 9/9 [00:00<00:00, 199.53it/s]


## Estimate EXPONENTIAL ELECTRICITY SHARE Growth
#### (required for scenarios)

In [13]:
def exponential_model(x, a, b):
    return a * np.exp(b * x)

def estimate_exp_elec_share_growth(df_temp,index):

    elec_vals = df_temp.loc[index,[
                            "elec_share_2020",
                            "elec_share_2021",
                            "elec_share_2022",
                            "elec_share_2023",
                            ]].values
    
    years = np.arange(0,4)
    try:
        params, covariance = curve_fit(exponential_model, years, elec_vals, maxfev=10000)
        elec_exp_intercept, elec_exp_growth = params
    except:
        elec_exp_intercept, elec_exp_growth = np.nan, np.nan
    
    return elec_exp_intercept, elec_exp_growth

for i in tqdm(range(len(df_filtered))):
    elec_exp_intercept, elec_exp_growth = estimate_exp_elec_share_growth(df_filtered,i)
    df_filtered.loc[i,"elec_rate"] = elec_exp_growth
    df_filtered.loc[i,"elec_rate_initial_value"] = elec_exp_intercept

100%|██████████| 9/9 [00:00<00:00, 80.10it/s]


In [14]:
df_filtered.to_csv("results/energy_consumption_firms_2020-2023.csv",index=False)